# Introduction to the Algorithm

This algorithm combines Ant Colony Optimization (ACO) with Active Learning (AL) to efficiently select a subset of candidates that best approximate a global covariance matrix, while minimizing computational costs. The process is structured as follows:

1. **Initial Exploration with ACO**:  
    Candidate selection is guided by two factors:  
    - **Pheromone (τ(λ))**: Encourages exploitation by favoring choices that have previously shown promise.
    - **Heuristic (η(λ))**: Drives exploration by estimating the variability using a surrogate model S(λ), indicating how much the solution could change in that region.
    The sampling probability for each candidate λ is proportional to τ(λ)^α ⋅ η(λ)^β, where α and β balance exploitation and exploration. A population of n_ants candidates is sampled according to this distribution.

2. **Rapid Candidate Filtering**:  
    For each sampled candidate λ_k, the surrogate quickly recalculates η(λ_k) = S(λ_k), updating the heuristic. A top-k selection is then performed to retain only the k candidates with the highest η(λ_k), focusing computational resources on the most promising options.

3. **Approximation of Error Reduction (Δ_k)**:  
    For each top-k candidate, the potential reduction in covariance error is estimated as:  
    Δ_k ≈ ||C_ref − C_e(Selected ∪ {λ_k})||_F − ||C_ref − C_e(Selected)||_F  
    This quantifies the improvement in global covariance accuracy if λ_k is added to the Selected set. The surrogate is used to estimate this difference, avoiding expensive exact evaluations.

4. **Exact Evaluation and Update**:  
    Only for the most promising top-k candidates is the exact value ||C(λ_k)|| computed. Pheromone values τ(λ_k) are updated based on these results, reinforcing the likelihood of selecting λ_k in future iterations. Selected candidates are added to the Selected set, and the surrogate model S(λ) is periodically retrained to maintain heuristic accuracy.

5. **Continuous Feedback between ACO and Active Learning**:  
    Each iteration integrates exploration (ACO) and learning (AL) in a feedback loop. The error ||C_ref − C_e|| progressively decreases as the Selected set is iteratively improved, creating a dynamic balance between speed (surrogate) and accuracy (exact evaluations).

6. **Final Outcome**:  
    At the end of the process, a Selected set of size m ≪ N is obtained, which closely approximates the global covariance matrix C. This approach drastically reduces computational costs while maintaining high estimation quality.§

In [ ]:
# Inizializzazione delle variabili
# τ(λ): dizionario dei feromoni per ogni candidato λ in Λ
tau = {lmbda: 1.0 for lmbda in Lambda}

# η(λ): dizionario delle euristiche calcolate tramite il surrogato S(λ)
eta = {lmbda: S(lmbda) for lmbda in Lambda}

# Selected: insieme dei candidati selezionati
Selected = set()

# C_e: matrice di covarianza stimata (inizialmente indefinita)
C_e = None

# val_count: conteggio delle valutazioni esatte
val_count = 0

In [ ]:
while val_count < budget:
    # Calcola errore corrente
    if C_e is not None:
        errore = np.linalg.norm(C_ref - C_e, ord='fro')
        if errore < epsilon:
            break
    else:
        errore = float('inf')

    # 1. Campionamento candidati con ACO
    prob = np.array([tau[lmbda]**alpha * eta[lmbda]**beta for lmbda in Lambda])
    prob = prob / prob.sum()
    candidates = np.random.choice(list(Lambda), size=n_ants, replace=False, p=prob)
    
    # 2. Ricalcola η(λ) per candidati usando il surrogato
    for lmbda in candidates:
        eta[lmbda] = S(lmbda)
    
    # 3. Calcola Δ_k per ogni candidato (stima riduzione errore)
    delta = {}
    for lmbda in candidates:
        # Stima C_e(Selected ∪ {lmbda}) tramite surrogato
        C_e_new = surrogate_covariance(Selected | {lmbda})
        delta[lmbda] = np.linalg.norm(C_ref - C_e_new, ord='fro') - \
                       (np.linalg.norm(C_ref - C_e, ord='fro') if C_e is not None else 0)
    
    # 4. Selezione top-k candidati da valutare esattamente
    scores = {lmbda: omega * delta[lmbda] + (1 - omega) * eta[lmbda] for lmbda in candidates}
    top_candidates = sorted(scores, key=scores.get, reverse=True)[:top_k]
    
    # 5. Valutazione esatta e aggiornamento
    for lmbda in top_candidates:
        x_star = exact_evaluation(lmbda)
        C_lmbda = compute_covariance(x_star)
        Selected.add(lmbda)
        C_e = empirical_covariance(Selected)
        tau[lmbda] += np.linalg.norm(C_lmbda)
        eta[lmbda] = np.linalg.norm(C_lmbda)
        val_count += 1
    
    # 6. Retraining surrogato
    if val_count % retrain_threshold == 0:
        S = retrain_surrogate(Selected, archive)